# Deep Learning for Deciphering Traffic Signs
_________________________________________________________________________________________________________________________________________________________________________________

##### Contributors:
 Victor Floriano, Yifan Fan, Jose Salerno

## Problem Statement & Motivation
As the world advances towards autonomous vehicles, our team has observed the remarkable efforts of large car manufacturers, who are working with data scientists to develop fully autonomous cars. Our team is excited to contribute to the development of this technology by creating a neural network model that will be able to classify different traffic signs. Our ultimate goal is to assist car makers in overcoming the challenges they may face in implementing neural network models that effectively read traffic signs and further their efforts toward a fully autonomous car or assisted driving. We believe autonomous driving to be an important problem to solve due to the great economic benefits it can generate for car manufacturers and the improvement of general driving safety.

## Data Preparation
 We've selected the German Traffic Sign Recognition Benchmark (GTSRB) as our primary dataset. It's renowned for its complexity, featuring over 50,000 images across more than 40 classes of traffic signs. The GTSRB is publicly accessible through two resources. To efficiently manage the extensive and complex GTSRB dataset, our strategy integrates preprocessing for uniformity, data augmentation for robustness, and batch processing for computational efficiency. We'll employ distributed computing to parallelize operations, enhancing processing speed, and use stratified sampling for quick experimentation without compromising representativeness.



---





# Project Breakdown Summary
1. EDA
3. Model Creation
    - MLP Models
    - CNN Model
    - Pre-trained Model
    - SVM
4. Results
5. Future Work
6. Conclusions
7. Sources

# 1. Exploratory Data Analysis

## Importing Packages

In [18]:
import torch
from torch.utils.data import Dataset
from torchvision import datasets, transforms
from torchvision.transforms import ToTensor
import matplotlib.pyplot as plt
import numpy as np
import pandas as pd
import os
from torch.utils.data import DataLoader, random_split
import torch.nn as nn
import torch.optim as optim

In [19]:
# from google.colab import drive
# drive.mount('/content/drive')

In [20]:
!pip install wandb -qU

In [21]:
enable_wandb = True
use_gpu = True
gpu_available = torch.cuda.iSalernos_available()
gpu_available

AttributeError: module 'torch.cuda' has no attribute 'iSalernos_available'

In [22]:
!pip install kaggle

Defaulting to user installation because normal site-packages is not writeable


In [23]:
import os
os.environ['PATH'] += ":/usr4/ba820/jsale017/.local/bin"

In [24]:
!kaggle datasets download -d meowmeowmeowmeowmeow/gtsrb-german-traffic-sign


Dataset URL: https://www.kaggle.com/datasets/meowmeowmeowmeowmeow/gtsrb-german-traffic-sign
License(s): CC0-1.0
gtsrb-german-traffic-sign.zip: Skipping, found more recently modified local copy (use --force to force download)


In [25]:
if enable_wandb:
  !pip install wandb -qU
  import wandb
  wandb.login()

In [26]:
!pip install tensorflow

Defaulting to user installation because normal site-packages is not writeable


In [27]:
from tensorflow.keras.preprocessing.image import ImageDataGenerator


train_datagen = ImageDataGenerator(
    rescale=1./255, 
    rotation_range=20,
    height_shift_range=0.2,
    validation_split=0.2  
)

train_data_dir = 'Train'

train_generator = train_datagen.flow_from_directory(
    train_data_dir,
    target_size=(56, 56),
    batch_size=32,
    class_mode='categorical',
    subset='training'
)

validation_generator = train_datagen.flow_from_directory(
    train_data_dir,
    target_size=(56, 56),
    batch_size=32,
    class_mode='categorical',
    subset='validation'
)


Found 31368 images belonging to 43 classes.
Found 7841 images belonging to 43 classes.


# 2. Model Creation 

## Sequential Model

In [28]:
from tensorflow import keras
from keras.models import Sequential
from keras.layers import Dense, Conv2D, MaxPooling2D, Flatten, Dropout

In [29]:
def MLP():
    model = Sequential()
    model.add(Flatten(input_shape=(56, 56, 3)))
    model.add(Dense(256, activation='relu',))
    model.add(Dropout(0.5))
    model.add(Dense(128, activation='relu'))
    model.add(Dropout(0.5))
    model.add(Dense(64, activation='relu'))
    model.add(Dense(43, activation='softmax'))
    return model

In [30]:
model = MLP()
model.summary()

/usr4/ba820/jsale017/.local/lib/python3.10/site-packages/keras/src/layers/reshaping/flatten.py:37: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(**kwargs)


Model: "sequential_1"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━┓
┃ Layer (type)                    ┃ Output Shape           ┃       Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━┩
│ flatten_1 (Flatten)             │ (None, 9408)           │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_4 (Dense)                 │ (None, 256)            │     2,408,704 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dropout_2 (Dropout)             │ (None, 256)            │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_5 (Dense)                 │ (None, 128)            │        32,896 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dropout_3 (Dropout)             │ (None, 128)            │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_6 (Dense)                 │ (None, 64)             │         8,256 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_7 (Dense)                 │ (None, 43)             │         2,795 │
└─────────────────────────────────┴────────────────────────┴───────────────┘

 Total params: 2,452,651 (9.36 MB)

 Trainable params: 2,452,651 (9.36 MB)

 Non-trainable params: 0 (0.00 B)

In [31]:
model.compile(optimizer='adam', loss='categorical_crossentropy', metrics=['accuracy'])

In [34]:
from tensorflow.keras.callbacks import EarlyStopping
model.fit(train_generator, validation_data = validation_generator, epochs = 10)

Epoch 1/10
981/981 ━━━━━━━━━━━━━━━━━━━━ 63s 63ms/step - accuracy: 0.0486 - loss: 3.7016 - val_accuracy: 0.0659 - val_loss: 3.4514
Epoch 2/10
981/981 ━━━━━━━━━━━━━━━━━━━━ 58s 59ms/step - accuracy: 0.0573 - loss: 3.4669 - val_accuracy: 0.0696 - val_loss: 3.4112
Epoch 3/10
981/981 ━━━━━━━━━━━━━━━━━━━━ 58s 59ms/step - accuracy: 0.0690 - loss: 3.4335 - val_accuracy: 0.0911 - val_loss: 3.3042
Epoch 4/10
981/981 ━━━━━━━━━━━━━━━━━━━━ 59s 60ms/step - accuracy: 0.0772 - loss: 3.4070 - val_accuracy: 0.1145 - val_loss: 3.2646
Epoch 5/10
981/981 ━━━━━━━━━━━━━━━━━━━━ 60s 61ms/step - accuracy: 0.0820 - loss: 3.3779 - val_accuracy: 0.0889 - val_loss: 3.3759
Epoch 6/10
981/981 ━━━━━━━━━━━━━━━━━━━━ 59s 59ms/step - accuracy: 0.0792 - loss: 3.3749 - val_accuracy: 0.1090 - val_loss: 3.2448
Epoch 7/10
981/981 ━━━━━━━━━━━━━━━━━━━━ 59s 60ms/step - accuracy: 0.0818 - loss: 3.3699 - val_accuracy: 0.0847 - val_loss: 3.2876
Epoch 8/10
981/981 ━━━━━━━━━━━━━━━━━━━━ 59s 60ms/step - accuracy: 0.0830 - loss: 3.3706 - 

## Keras CNN Model

In [47]:
def MLP():
    model = Sequential()
    model.add(Conv2D(32, kernel_size=(4,4), activation='relu', input_shape=(56, 56, 3)))
    model.add(Dropout(0.5))
    model.add(Flatten())
    model.add(Dense(128, activation='relu'))
    model.add(Dropout(0.5))
    model.add(Dense(64, activation='relu'))  # Adjusted number of units
    model.add(Dropout(0.5))
    model.add(Dense(43, activation='softmax'))
    return model

In [48]:
cnn_model = MLP()
cnn_model.summary()

Model: "sequential_5"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━┓
┃ Layer (type)                    ┃ Output Shape           ┃       Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━┩
│ conv2d_3 (Conv2D)               │ (None, 53, 53, 32)     │         1,568 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dropout_13 (Dropout)            │ (None, 53, 53, 32)     │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ flatten_6 (Flatten)             │ (None, 89888)          │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_17 (Dense)                │ (None, 128)            │    11,505,792 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dropout_14 (Dropout)            │ (None, 128)            │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_18 (Dense)                │ (None, 64)             │         8,256 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dropout_15 (Dropout)            │ (None, 64)             │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_19 (Dense)                │ (None, 43)             │         2,795 │
└─────────────────────────────────┴────────────────────────┴───────────────┘

 Total params: 11,518,411 (43.94 MB)

 Trainable params: 11,518,411 (43.94 MB)

 Non-trainable params: 0 (0.00 B)

In [49]:
cnn_model.compile(optimizer='adam', loss='categorical_crossentropy', metrics=['accuracy'])

In [ ]:
from tensorflow.keras.callbacks import EarlyStopping
callback = EarlyStopping(monitor='val_loss', patience=3)
cnn_model.fit(train_generator, validation_data = validation_generator, epochs = 35, callbacks = [callback])

Epoch 1/35
981/981 ━━━━━━━━━━━━━━━━━━━━ 96s 97ms/step - accuracy: 0.0774 - loss: 3.6300 - val_accuracy: 0.2913 - val_loss: 2.7149
Epoch 2/35
981/981 ━━━━━━━━━━━━━━━━━━━━ 75s 76ms/step - accuracy: 0.2172 - loss: 2.7372 - val_accuracy: 0.3400 - val_loss: 2.2085
Epoch 3/35
981/981 ━━━━━━━━━━━━━━━━━━━━ 81s 82ms/step - accuracy: 0.2676 - loss: 2.4412 - val_accuracy: 0.3654 - val_loss: 2.0344
Epoch 4/35
981/981 ━━━━━━━━━━━━━━━━━━━━ 91s 92ms/step - accuracy: 0.2990 - loss: 2.2662 - val_accuracy: 0.3758 - val_loss: 1.9954
Epoch 5/35
981/981 ━━━━━━━━━━━━━━━━━━━━ 87s 88ms/step - accuracy: 0.3171 - loss: 2.2071 - val_accuracy: 0.3719 - val_loss: 1.9481
Epoch 6/35
981/981 ━━━━━━━━━━━━━━━━━━━━ 82s 83ms/step - accuracy: 0.3213 - loss: 2.1841 - val_accuracy: 0.3714 - val_loss: 1.9173
Epoch 7/35
981/981 ━━━━━━━━━━━━━━━━━━━━ 82s 83ms/step - accuracy: 0.3307 - loss: 2.1342 - val_accuracy: 0.3746 - val_loss: 1.9230
Epoch 8/35
981/981 ━━━━━━━━━━━━━━━━━━━━ 84s 86ms/step - accuracy: 0.3370 - loss: 2.1106 - 

## Support Vector Machine Model

# 3. Results

# 4. Future Work

# 5. Conclusions

# 6. Sources

## Sources:

- https://blog.keras.io/building-powerful-image-classification-models-using-very-little-data.html